In [ ]:
import random
import pandas as pd

# Load the dataset
dataset = pd.read_csv('../csv/DataSet_Hashed.csv')
print (dataset.head)

In [ ]:
len(dataset)

#First phase of coding

In [ ]:
import pandas as pd
import requests
import json
import os
import time
from dotenv import load_dotenv

# --- MODIFIED: Hardened, Multi-Stage JSON Extraction Function ---
def extract_json_from_text(text):
    """
    Finds and loads the first valid JSON object from a string.
    Priority 1: Look for content within <json_output> delimiters.
    Priority 2: Fallback to finding the outermost JSON object {}.
    """
    try:
        # Priority 1: Look for content within <json_output> tags
        start_tag = "<json_output>"
        end_tag = "</json_output>"
        tag_start_index = text.find(start_tag)
        tag_end_index = text.rfind(end_tag)

        if tag_start_index != -1 and tag_end_index != -1:
            json_str = text[tag_start_index + len(start_tag):tag_end_index].strip()
            return json.loads(json_str)

        # Priority 2: Fallback to finding the first and last curly brace
        start_index = text.find('{')
        end_index = text.rfind('}')
        if start_index != -1 and end_index != -1:
            json_str = text[start_index:end_index+1]
            return json.loads(json_str)
            
    except json.JSONDecodeError as e:
        # This error message is crucial for debugging.
        print(f"    Could not parse JSON from response. Error: {e}")
        print(f"    Raw Text Received: {text}") # Print the raw text that failed
        return None
    
    # If no JSON is found at all, return None
    return None

# --- MODIFIED: Core analysis function with retry logic and hardened prompt ---
def analyze_tiktok_video(row, headers):
    """
    Analyzes a single row of TikTok data with a hardened prompt and retry logic.
    """
    codebook = """
    
   ## Codebook of the Qualitative Content Analysis ##

    **Category: Journey Narratives**
    - Code: JN-TE, Label: Transformative Experiences, Description: Self-actualization through travel.
    - Code: JN-EP, Label: Episodic Progress, Descrition: Narrative of experiences, milestones while traveling.
    - Code: JN-PG, Label: Personal Growth, Description: Insights and achievements through travel.

    **Category: Digital Representation**
    - Code: DR-VS, Label: Visual Storytelling, Description: Emphasize visual appeal, e.g., coastal landscapes, city lines, and airplane flights.
    - Code: DR-IE, Label: Idealization of Experiences, Description: The idealized version of the future, career, and destinations.
    - Code: DR-CA, Label: Cultural Authenticity, Description: The code showcases local experiences, cultural appropriation, or superficial representation.
    - Code: DR-PSC, Label: Platform-Specific Content, Description: Content formatted according to TikTok grammar and logic.

    **Category: Commercialization**
    - Code: CA-TC, Label: Tourism and Commerce, Description: Destination promotion content may contribute to over-tourism issues.
    - Code: CA-PC, Label: Promotional Content, Description: Affiliated links or product placement in the content.

    **Category: Symbolism**
    - Code: SY-SL, Label: Symbolic Imagery, Description: Symbolic representation: e.g., women in bikinis represent leisure activities.
    - Code: SY-RB, Label: Rituals, Description: The code represents local traditions, e.g., Muslim broadcasting of psalms at 6 a.m.

    **Category: Existential Themes**
    - Code: ET-SD, Label: Self-Discovery, Description: Knowledge acquisition, discovery.
    - Code: ET-PM, Label: Purpose and Meaning, Description: Finding a meaning in the activity.
    - Code: ET-SG, Label: Spiritual Growth, Description: The notion of becoming a better version of oneself.
    - Code: ET-IC, Label: Identity Construction, Description: Narratives that aim to construct identity online.
    - Code: ET-PF, Label: Personal & Spatial Freedom, Description: The freedom through mobility, the perception that the whole world is open for exploration.

    **Category: Hedonism**
    - Code: HE-SA, Label: Sport Activities, Description: The active type of recreational consumption, e.g., surfing, cycling, paragliding.
    - Code: HE-CE, Label: Culinary Experiences, Description: Projection of food consumption, preparation, or purchase.
    - Code: HE-SE, Label: Shared Cultural Experiences & Bonding, Description: Experiences of pleasure and sensory gratification include festivals, carnivals, and performances.

    **Category: TikTok Affordances**
    - Code: TA-AC, Label: Accessibility (e.g., templates), Description: Ease of expression and re-use of content.
    - Code: TA-DI, Label: Discoverability (e.g., hashtags, For You Page, audios), Description: Features that enhance content discoverability, enabling the reach of wider audiences beyond immediate followers.
    - Code: TA-RL, Label: Relatability (trends, challenges), Description: Dialogic content that resonates with and encourages viewer self-identification and participation.
    """
    
    # The system prompt is now highly structured to guide the model.
    system_message_content = f"""
    You are a qualitative coder. Your task is to analyze the provided TikTok data and return a single, valid JSON object.

    **Instructions:**
    1.  Carefully review the Content Description, Hashtags, and Captions below.
    2.  Based on the Codebook provided, determine the most relevant codes (up to 5).
    3.  Formulate a brief `reason` that explains and justifies your code choices.
    4.  Your final output MUST be ONLY a valid JSON object enclosed within `<json_output>` and `</json_output>` delimiters. Do NOT include any other text, markdown, or explanations like `<think>` outside these delimiters.

    **Codebook:**
    {codebook}
    """
    
    # The user prompt uses the "sandwich" technique, placing the data between instructions.
    user_prompt = f"""
    Analyze the following data based on the system instructions:

    **Input Data:**
    - Content Description: {row.get('desc', 'N/A')}
    - Hashtags: {row.get('hashtagNames', 'N/A')}
    - Captions: {row.get('stickersText', 'N/A')}

    Remember, provide your response ONLY as a JSON object wrapped in <json_output> tags.
    """

    messages = [{"role": "system", "content": system_message_content}, {"role": "user", "content": user_prompt}]
    payload = {"model": "sonar-reasoning", "messages": messages, "max_tokens": 2048, "temperature": 0.1}

    # The robust retry logic remains.
    max_retries = 3
    base_delay = 2

    for attempt in range(max_retries):
        try:
            response = requests.post("https://api.perplexity.ai/chat/completions", headers=headers, json=payload, timeout=60)

            if response.status_code == 200:
                raw_output = response.json()['choices'][0]['message']['content']
                return extract_json_from_text(raw_output)
            
            elif response.status_code == 429:
                delay = (attempt + 1) * 10
                print(f"    Rate limit hit. Waiting {delay}s before retrying...")
                time.sleep(delay)
            elif 500 <= response.status_code < 600:
                delay = base_delay * (2 ** attempt)
                print(f"    Server Error {response.status_code}. Retrying in {delay}s...")
                time.sleep(delay)
            else:
                print(f"    Client/API Error {response.status_code}: {response.text}")
                return None

        except requests.exceptions.RequestException as e:
            delay = base_delay * (2 ** attempt)
            print(f"    Request Exception: {e}. Retrying in {delay}s...")
            time.sleep(delay)

    print(f"    All {max_retries} retries failed for Video ID {row.get('hashed_videoId', 'N/A')}.")
    return None

# --- Main execution block with CHECKPOINTING ---
if __name__ == "__main__":
    load_dotenv()
    sonar_key = os.getenv("SONAR_KEY")
    if not sonar_key: raise ValueError("Error: SONAR_KEY not found in .env file.")
    
    print("SONAR_KEY loaded successfully.")
    headers = {"Authorization": f"Bearer {sonar_key}", "Content-Type": "application/json"}
    
    output_json_file = 'analysis_results.json'
    failed_ids_file = 'failed_videos.txt'
    
    try:
        text_cols = ['desc', 'hashtagNames', 'stickersText']
        for col in text_cols:
            if col in dataset.columns:
                dataset[col] = dataset[col].fillna('')
        
       # Checkpointing logic to resume from where you left off
        successful_results = []
        processed_ids = set()

        if os.path.exists(output_json_file):
            print(f"Found existing results file: '{output_json_file}'. Resuming...")
            try:
                with open(output_json_file, 'r', encoding='utf-8') as f:
                    successful_results = json.load(f)
                    processed_ids = {item['hashed_videoId'] for item in successful_results if 'hashed_videoId' in item}
                print(f"Loaded {len(successful_results)} previously processed videos.")
            except (json.JSONDecodeError, TypeError):
                print("Warning: Could not read existing results file. Starting from scratch.")
                successful_results = []
                processed_ids = set()

        all_rows = dataset.to_dict(orient='records')
        rows_to_process = [row for row in all_rows if row['hashed_videoId'] not in processed_ids]
        
        if not rows_to_process:
            print("All videos have already been processed. Exiting.")
        else:
            print(f"\n--- Starting analysis on {len(rows_to_process)} remaining videos ---")
            
            for i, row in enumerate(rows_to_process):
                print(f"Processing {i+1}/{len(rows_to_process)} - Video ID: {row['hashed_videoId']}")
                result = analyze_tiktok_video(row, headers)
                
                if result:
                    result['hashed_videoId'] = row['hashed_videoId']
                    successful_results.append(result)
                    # Checkpoint every 50 results
                    if (i + 1) % 50 == 0:
                        print(f"    Checkpointing... saving {len(successful_results)} results.")
                        with open(output_json_file, 'w', encoding='utf-8') as f:
                            json.dump(successful_results, f, ensure_ascii=False, indent=4)
                else:
                    print(f"    *** FAILED to process Video ID: {row['hashed_videoId']} ***")
                    with open(failed_ids_file, 'a', encoding='utf-8') as f:
                        f.write(f"{row['hashed_videoId']}\n")
                        %store failed_ids_file

        print("\n--- Finalizing Results ---")
        with open(output_json_file, 'w', encoding='utf-8') as f:
            json.dump(successful_results, f, ensure_ascii=False, indent=4)
            
        print(f"\n--- Script finished ---")
        print(f"Total successfully processed: {len(successful_results)} videos.")

    except Exception as e:
        print(f"An unexpected error occurred during main execution: {e}")


#Second phase of coding

In [ ]:
import json
import os
import requests
from dotenv import load_dotenv

def extract_json_from_text(text):
    """
    Finds and loads the first valid JSON object from a string,
    looking for content within <json_output> delimiters if present.
    Falls back to curly brace matching if delimiters are not found or invalid.
    """
    start_tag = "<json_output>"
    end_tag = "</json_output>"
    tag_start_index = text.find(start_tag)
    tag_end_index = text.rfind(end_tag)

    if tag_start_index != -1 and tag_end_index != -1:
        json_str_candidate = text[tag_start_index + len(start_tag):tag_end_index].strip()
        try:
            return json.loads(json_str_candidate)
        except json.JSONDecodeError:
            pass
    
    # Priority 2: Fallback to finding the first and last curly brace
    try:
        start_index = text.find('{')
        end_index = text.rfind('}')
        if start_index != -1 and end_index != -1:
            json_str = text[start_index:end_index+1]
            return json.loads(json_str)
    except (json.JSONDecodeError, IndexError) as e:
        print(f"Could not parse JSON from response: {text}, Error: {e}")
        return None
    return None

def analyze_tiktok_video_element(row, headers):
    """
    Performs second-step coding using a defined framework and a dedicated LLM.
    Makes direct API calls using requests with provided headers.
    """
    system_message_content = """
    You are a qualitative coder performing a second-order classification. Your task is to analyze a set of pre-assigned codes from a TikTok video and map them to a single 'Element' and its corresponding 'Primary Need' from the provided framework. 
    Important: Regardless of the input language, acknowledge the language but analyze in English using the same framework.

    **## THEORETICAL FRAMEWORK ##**

    You must use the following Hierarchy of Needs for each element:

    {
        "Worker": {"Basic": "Location independence, internet connection", "Safety": "Diversification of income, stable work environment", "Social": "Professional networking, co-working spaces", "Esteem": "Career growth, skill development", "Self-actualization": "Work-life balance, meaningful work"},
        "Tourist": {"Basic": "Accommodation, food", "Safety": "Comfortable and affordable travel", "Social": "Meeting locals and other travelers", "Esteem": "Unique experiences, cultural immersion", "Self-actualization": "Personal growth through travel"},
        "Migrant": {"Basic": "Legal status, housing", "Safety": "Understanding local laws, healthcare access", "Social": "Integration into the local community", "Esteem": "Cultural adaptation, language skills", "Self-actualization": "Sense of belonging in a new place"},
        "Pilgrim": {"Basic": "Finding purpose in travel", "Safety": "Spiritual or emotional security", "Social": "Connecting with like-minded individuals", "Esteem": "Personal transformation", "Self-actualization": "Achieving life goals, enlightenment"}
    }

    **## YOUR TASK ##**

    Based on the input codes provided by the user, produce a single JSON object as output. Your output MUST be enclosed within `<json_output>` and `</json_output>` delimiters.

    The JSON must contain:
    - "reasoning": A concise explanation for why you chose the element, based on the input codes.
    - "element": One of the four elements (Worker, Tourist, Migrant, Pilgrim).
    - "need": The primary need (Basic, Safety, Social, Esteem, Self-actualization) demonstrated in the content.
    """
    
    user_prompt = f"""
    Analyze the following codes and provide your classification.
    Reason: {row.get('reason', 'N/A')}
    Input Codes: {row.get('codes', 'N/A')}
    Video ID: {row.get('hashed_videoId', 'N/A')}
    """

    messages = [
        {"role": "system", "content": system_message_content},
        {"role": "user", "content": user_prompt}
    ]

    payload = {
        "model": "sonar-reasoning",
        "messages": messages,
        "max_tokens": 2048,
        "temperature": 0.2
    }

    
    # The robust retry logic remains.
    max_retries = 3
    base_delay = 2
    for attempt in range(max_retries):
        try:
            response = requests.post("https://api.perplexity.ai/chat/completions", headers=headers, json=payload, timeout=60)

            if response.status_code == 200:
                raw_output = response.json()['choices'][0]['message']['content']
                return extract_json_from_text(raw_output)
            
            elif response.status_code == 429:
                delay = (attempt + 1) * 10
                print(f"    Rate limit hit. Waiting {delay}s before retrying...")
                time.sleep(delay)
            elif 500 <= response.status_code < 600:
                delay = base_delay * (2 ** attempt)
                print(f"    Server Error {response.status_code}. Retrying in {delay}s...")
                time.sleep(delay)
            else:
                print(f"    Client/API Error {response.status_code}: {response.text}")
                return None

        except requests.exceptions.RequestException as e:
            delay = base_delay * (2 ** attempt)
            print(f"    Request Exception: {e}. Retrying in {delay}s...")
            time.sleep(delay)

    print(f"    All {max_retries} retries failed for Video ID {row.get('hashed_videoId', 'N/A')}.")
    return None

def process_analysis_results(file_path):
    """
    Reads the analysis_results.json file and returns the data as a list of dictionaries.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return data
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return []
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {file_path}.")
        return []
    except Exception as e:
        print(f"Unexpected error while processing {file_path}: {e}")
        return []

# --- Main execution block with CHECKPOINTING ---
if __name__ == "__main__":
    load_dotenv()
    sonar_key = os.getenv("SONAR_KEY")
    if not sonar_key: raise ValueError("Error: SONAR_KEY not found in .env file.")
    
    print("SONAR_KEY loaded successfully.")
    headers = {"Authorization": f"Bearer {sonar_key}", "Content-Type": "application/json"}
    
    output_json_file = 'needs_results.json'
    failed_ids_file = 'failed_needs_ids.txt'
    
    try:
        # Checkpointing logic to resume from where you left off
        successful_results = []
        processed_ids = set()
        
        if os.path.exists(output_json_file):
            print(f"Found existing results file: '{output_json_file}'. Resuming...")
            try:
                with open(output_json_file, 'r', encoding='utf-8') as f:
                    successful_results = json.load(f)
                    processed_ids = {item['hashed_videoId'] for item in successful_results if 'hashed_videoId' in item}
                print(f"Loaded {len(successful_results)} previously processed videos.")
            except (json.JSONDecodeError, TypeError):
                print("Warning: Could not read existing results file. Starting from scratch.")
                successful_results = []
                processed_ids = set()
        
        # Load the dataset from analysis_results.json
        all_rows = process_analysis_results('analysis_results.json')
        rows_to_process = [row for row in all_rows if row.get('hashed_videoId', 'N/A') not in processed_ids]
        
        if not rows_to_process:
            print("All videos have already been processed. Exiting.")
        else:
            print(f"\n--- Starting analysis on {len(rows_to_process)} remaining videos ---")
            
            for i, row in enumerate(rows_to_process):
                print(f"Processing {i+1}/{len(rows_to_process)} - Video ID: {row.get('hashed_videoId', 'N/A')}")
                result = analyze_tiktok_video_element(row, headers)
                
                if result:
                    result['hashed_videoId'] = row.get('hashed_videoId', 'N/A')
                    successful_results.append(result)
                    # Checkpoint every 50 results
                    if (i + 1) % 50 == 0:
                        print(f"    Checkpointing... saving {len(successful_results)} results.")
                        with open(output_json_file, 'w', encoding='utf-8') as f:
                            json.dump(successful_results, f, ensure_ascii=False, indent=4)
                else:
                    print(f"    *** FAILED to process Video ID: {row.get('hashed_videoId', 'N/A')} ***")
                    with open(failed_ids_file, 'a', encoding='utf-8') as f:
                        f.write(f"{row.get('hashed_videoId', 'N/A')}\n")
        
        print("\n--- Finalizing Results ---")
        with open(output_json_file, 'w', encoding='utf-8') as f:
            json.dump(successful_results, f, ensure_ascii=False, indent=4)
            
        print(f"\n--- Script finished ---")
        print(f"Total successfully processed: {len(successful_results)} videos.")
    
    except Exception as e:
        print(f"An unexpected error occurred during main execution: {e}")

#analysis if failed ID's

In [ ]:
import pandas as pd
import json

# Hardcoded list of the 4 still failing video IDs
failed_video_ids = ['XXX','YYY','ZZZ'] #put your failed hashed IDS

# Load analysis_results.json
with open('analysis_results.json', 'r', encoding='utf-8') as f:
    analysis_results = json.load(f)

# Filter entries for failed video IDs from analysis_results
failed_analysis_results = [item for item in analysis_results if str(item.get('videoId', '')) in failed_video_ids]

# Filter entries for failed video IDs from dataset
failed_dataset_entries = dataset[dataset['videoId'].isin(failed_video_ids)].to_dict(orient='records')

print("\nFailed analysis_results entries:")
for entry in failed_analysis_results:
    print(entry)

print("\nFailed dataset entries:")
for entry in failed_dataset_entries:
    print(entry)
